In [ ]:
import os, json, math, textwrap, ast
import numpy as np
import pandas as pd
import streamlit as st

# # Optional GPU accel; safe to ignore if not available
# try:
#     import cuml.accel
#     cuml.accel.install()
#     print("RAPIDS acceleration enabled.")
# except Exception as e:
#     print("RAPIDS accel not available; CPU mode.", e)

##from umap import UMAP
##from hdbscan import HDBSCAN

from snowflake.snowpark.context import get_active_session
from snowflake.core import Root

session = get_active_session()
root = Root(session)

print("✅ Snowpark session ready.")



In [ ]:
# ==== SOURCE DATA ====
# Expected schema (minimum): SURVEY_ID STRING, KEYWORDS STRING(JSON of {"keywords": ["...","..."]})
SOURCE_TABLE = "SURVEY_KEY_TOPICS"   # <- change if different

# ==== INTERMEDIATE / OUTPUT TABLES ====
SURVEY_KEYWORDS_EXPLODED = "SURVEY_KEYWORDS_EXPLODED"      # SURVEY_ID, KEYWORD
KEYWORD_EMBEDDINGS       = "KEYWORD_EMBEDDINGS"            # KEYWORD, EMBEDDING (ARRAY/FLOAT)
KEYWORD_CLUSTERS         = "KEYWORD_CLUSTERS"              # KEYWORD, CLUSTER_ID, CLUSTER_LABEL
CLUSTER_DICTIONARY       = "CLUSTER_DICTIONARY"            # CLUSTER_ID, CLUSTER_LABEL, EXAMPLE_KEYWORDS, CLUSTER_SIZE
SURVEY_CLUSTER_OUTPUT    = "SURVEY_CLUSTER_OUTPUT"         # SURVEY_ID, KEYWORDS (original), CLUSTER_LABELS (ARRAY)

# ==== EMBEDDING ====
EMBED_MODEL      = "snowflake-arctic-embed-m-v1.5"
EMBED_BATCH_SIZE = 256       # Safe batch size; tune if you like
EMBED_DTYPE_NP   = np.float32

# ==== CLUSTERING ====
# We keep it simple: HDBSCAN on raw embeddings (UMAP optional). HDBSCAN finds K automatically.
USE_UMAP              = False  # Set True for nicer separation on large corpora
UMAP_N_COMPONENTS     = 5
UMAP_N_NEIGHBORS      = 15
UMAP_MIN_DIST         = 0.0
UMAP_METRIC           = "cosine"
HDBSCAN_MIN_CLUSTER_SIZE = None  # if None -> heuristic sqrt(n)
HDBSCAN_MIN_SAMPLES      = None  # let HDBSCAN infer
HDBSCAN_METRIC           = "euclidean"  # on UMAP/embedding space
OUTLIER_LABEL            = "Other"      # label for cluster -1

# ==== LABELING ====
LABEL_MODEL       = "snowflake-llama-3.3-70b"
MAX_LABEL_LEN     = 60
LABEL_TOP_K_WORDS = 12        # how many keywords to show the model per cluster


In [ ]:
-- pre-uploaded synthetic data
SELECT *
FROM DEMO_DB.CRM.PRODUCT_SURVEY
LIMIT 10;



## Extract and Embed Keywords/Aspects

In [ ]:
create or replace table SURVEY_KEY_TOPICS
as
SELECT r.survey_id, r.review_text,
AI_COMPLETE(
MODEL => 'CLAUDE-4-SONNET',
PROMPT => 'Extract 1-6 concise product aspects from this retail review.\n' ||
'Rules: short phrases (2-4 words), copy from text if possible, no sentiment words, no duplicates.\n' ||
'Example: Great quality-husband loves them. Really comfy and true to size. -> ["quality", "comfy", "size"]\n' ||
'Return ONLY a JSON array of strings.\n\n' || r.REVIEW_TEXT,
 response_format => {
    'type': 'json',
    'schema': {
      'type': 'object',
      'properties': {
        'keywords': {
          'type': 'array',
          'items': {'type': 'string'}
            }
        }
    }
}
,
MODEL_PARAMETERS => {
        'temperature': 0,
        'max_tokens': 4096
    }
) as KEYWORDS
FROM DEMO_DB.CRM.PRODUCT_SURVEY r
where 1=1
and REVIEW_TEXT != ''
--limit 5
;

In [ ]:
SELECT * FROM SURVEY_KEY_TOPICS limit 10;

In [ ]:
-- 3A) Start from a clean exploded table (no embeddings here)
CREATE OR REPLACE TABLE SURVEY_KEYWORDS_EXPLODED AS
SELECT
  s.SURVEY_ID,
  LOWER(TRIM(f.value::string)) AS KEYWORD
FROM SURVEY_KEY_TOPICS s,
LATERAL FLATTEN( INPUT => TRY_PARSE_JSON(s.KEYWORDS):keywords ) f
WHERE f.value IS NOT NULL
  AND TRY_PARSE_JSON(s.KEYWORDS) IS NOT NULL
  AND TRIM(f.value::string) <> '';

-- 3B) Make a DEDUPED list of unique keywords
CREATE OR REPLACE TEMP TABLE _UNIQUE_KEYWORDS AS
SELECT DISTINCT KEYWORD
FROM SURVEY_KEYWORDS_EXPLODED
WHERE KEYWORD IS NOT NULL AND KEYWORD <> '';

-- 3C) Embed ONCE per unique keyword 
CREATE OR REPLACE TABLE KEYWORD_EMBEDDINGS AS
SELECT
  KEYWORD,
  AI_EMBED('snowflake-arctic-embed-m-v1.5', KEYWORD) AS EMBEDDING
FROM _UNIQUE_KEYWORDS;



In [ ]:
select * from KEYWORD_EMBEDDINGS order by keyword desc limit 5;

## Cluster Analysis

In [ ]:
# Pull a deterministic, unique keyword list for clustering
kw_df = session.table("KEYWORD_EMBEDDINGS") \
               .select("KEYWORD", "EMBEDDING") \
               .sort("KEYWORD") \
               .to_pandas()

# Build a KEYWORD -> vector map (ensure float32)
embed_map = {
    row["KEYWORD"]: np.asarray(row["EMBEDDING"], dtype=np.float32)
    for _, row in kw_df.iterrows()
}

# The ordered keyword list used for clustering
keywords = kw_df["KEYWORD"].tolist()

# Build X in the SAME order
X = np.vstack([embed_map[k] for k in keywords]).astype(np.float32)

print("✅ Embedding matrix X:", X.shape, "| unique keywords:", len(keywords))


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# X, keywords are already built in Cell 3
n = len(keywords)
assert X.shape[0] == n, "X and keywords length mismatch."

# 1) L2-normalize (spherical k-means behavior; cosine-friendly)
X_norm = normalize(X.astype(np.float32), norm="l2", axis=1)

# 2) (Optional) PCA to denoise high-dim vectors when n is small
#    Keep enough components but not more than n-1 to avoid degenerate cases
USE_PCA = True
PCA_COMPONENTS = min(100, max(10, n - 1))  # between 10 and 100, capped by n-1

if USE_PCA:
    pca = PCA(n_components=PCA_COMPONENTS, random_state=42)
    X_work = pca.fit_transform(X_norm)
else:
    X_work = X_norm

# 3) Choose K automatically by silhouette over a small, sensible range
def choose_k(X_mat, k_min=4, k_max=12):
    ks = []
    scores = []
    for k in range(k_min, min(k_max, max(2, n - 1)) + 1):
        km = KMeans(n_clusters=k, random_state=42, n_init="auto")
        labels = km.fit_predict(X_mat)
        # cosine silhouette pairs well with normalized embeddings; for PCA space euclidean is fine too
        # If you used PCA, euclidean silhouette is consistent with KMeans objective:
        metric = "euclidean" if USE_PCA else "cosine"
        # Handle tiny clusters edge cases
        if len(set(labels)) < 2:
            continue
        try:
            score = silhouette_score(X_mat, labels, metric=metric)
            ks.append(k)
            scores.append(score)
        except Exception:
            # Can fail for pathological clusterings; skip
            pass
    if not scores:
        # Fallback: 5 clusters if nothing scored
        return 5, None
    best_idx = int(np.argmax(scores))
    return ks[best_idx], float(scores[best_idx])

k_opt, sil = choose_k(X_work, k_min=6, k_max=10)
print(f"🔎 Auto-selected K={k_opt} (silhouette={sil if sil is not None else 'n/a'})")

# 4) Final KMeans fit
kmeans = KMeans(n_clusters=k_opt, random_state=42, n_init="auto")
labels = kmeans.fit_predict(X_work)

# (Optional) get centroids back in original space for reference/labeling
# For PCA case, map centroids back to normalized space (approximate)
if USE_PCA:
    centroids_pca = kmeans.cluster_centers_
    centroids_norm = pca.inverse_transform(centroids_pca)
else:
    centroids_norm = kmeans.cluster_centers_

# Ensure centroids are unit-norm (useful for “nearest keyword” medoids)
centroids_norm = normalize(centroids_norm, norm="l2", axis=1)

# Metrics
n_clusters = len(set(labels))
sizes = pd.Series(labels).value_counts().sort_index()
print(f"📊 KMeans clusters: {n_clusters}")
print("Cluster sizes:\n", sizes.to_string())


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

# Inputs available from prior cells:
# - X (n x d) or X_work; we'll use X_norm for cosine sims (unit vectors)
# - labels: np.ndarray of shape (n,)
# - keywords: list[str] length n

# If you don't already have X_norm, create it:
try:
    X_norm  # noqa: F401
except NameError:
    X_norm = normalize(X.astype(np.float32), norm="l2", axis=1)

# --- knobs ---
MIN_SIM_TO_MERGE = 0.75  # if singleton's best cosine to any centroid >= this, merge; else keep singleton

# Helper: compute unit centroids per cluster
def centroids_from_labels(Xu, lbls):
    centroids = {}
    members = {}
    for cid in np.unique(lbls):
        idxs = np.where(lbls == cid)[0]
        members[int(cid)] = idxs
        c = Xu[idxs].mean(axis=0).astype(np.float32)
        c /= (np.linalg.norm(c) + 1e-12)
        centroids[int(cid)] = c
    return centroids, members

labels_before = labels.copy()
sizes_before = pd.Series(labels_before).value_counts().sort_index()
print("Before (sizes):\n", sizes_before.to_string())

# Compute centroids and find singleton clusters
C, members = centroids_from_labels(X_norm, labels_before)
singleton_ids = [cid for cid, idxs in members.items() if len(idxs) == 1]

if singleton_ids:
    # Build matrix of non-singleton centroids
    non_singletons = [cid for cid, idxs in members.items() if len(idxs) > 1]
    if non_singletons:  # only proceed if there is at least one non-singleton to merge into
        M = np.stack([C[cid] for cid in non_singletons], axis=0)  # rows are unit vectors
        M_ids = non_singletons

        labels_after = labels_before.copy()
        for cid in singleton_ids:
            idx = members[cid][0]        # the single member's row index
            v = X_norm[idx]              # unit vector for that keyword
            sims = M @ v                 # cosine similarity to each non-singleton centroid
            j = int(np.argmax(sims))
            if sims[j] >= MIN_SIM_TO_MERGE:
                # merge singleton into the nearest non-singleton cluster
                labels_after[idx] = M_ids[j]
            # else: keep as its own tiny cluster

    else:
        labels_after = labels_before  # nothing to merge into
else:
    labels_after = labels_before      # no singletons

sizes_after = pd.Series(labels_after).value_counts().sort_index()
print("\nAfter singleton-merge (sizes):\n", sizes_after.to_string())

# Use these going forward
final_labels = labels_after 

In [ ]:
# Keyword -> cluster dataframe
kw_clusters = pd.DataFrame({"KEYWORD": keywords, "CLUSTER_ID": final_labels})

# For each cluster, pick top K example keywords (by frequency in corpus)
# Here all keywords are unique strings; so we just take any K per cluster.
cluster_examples = (
    kw_clusters[kw_clusters["CLUSTER_ID"] != -1]
    .groupby("CLUSTER_ID")["KEYWORD"]
    .apply(lambda s: sorted(s.tolist())[:LABEL_TOP_K_WORDS])
    .reset_index(name="EXAMPLE_KEYWORDS")
)

cluster_sizes = (
    kw_clusters[kw_clusters["CLUSTER_ID"] != -1]
    .groupby("CLUSTER_ID")["KEYWORD"]
    .size()
    .reset_index(name="CLUSTER_SIZE")
)

cluster_dict = cluster_examples.merge(cluster_sizes, on="CLUSTER_ID", how="left")
print(f"Will label {len(cluster_dict)} clusters (excluding outliers).")
cluster_dict

In [ ]:
if not cluster_dict.empty:
    # Copy & prep
    tmp = cluster_dict.copy()

    # 🔧 Remove any leftover label columns from previous runs (avoids *_x, *_y growth)
    tmp = tmp.drop(columns=[c for c in tmp.columns if c.upper().startswith("CLUSTER_LABEL")], errors="ignore")

    # Build prompt inputs
    tmp["EXAMPLE_KEYWORDS_STR"] = tmp["EXAMPLE_KEYWORDS"].apply(lambda xs: ", ".join(xs))

    # Overwrite temp table cleanly
    session.sql("DROP TABLE IF EXISTS TMP_CLUSTER_DICT").collect()
    session.write_pandas(
        tmp[["CLUSTER_ID", "EXAMPLE_KEYWORDS_STR", "CLUSTER_SIZE"]],
        "TMP_CLUSTER_DICT",
        auto_create_table=True,
        overwrite=True,
    )

    # Label clusters with Cortex (or AI_COMPLETE)
    labeling_sql = f"""
    WITH P AS (
      SELECT 
        CLUSTER_ID,
        CONCAT(
          'You are a product quality assurance expert tasked with providing a concise label ',
          'to group feedback categories from reviews for lululemon clothing.',
          ' Given these example keywords: ',
          EXAMPLE_KEYWORDS_STR,
          ' — return a concise 1–3 word noun phrase that best names the shared theme/review topic.',
          ' Return ONLY the phrase, no punctuation or quotes.'
        ) AS PROMPT
      FROM TMP_CLUSTER_DICT
    )
    SELECT 
      CLUSTER_ID,
      AI_COMPLETE('{LABEL_MODEL}', PROMPT) AS RAW_LABEL
    FROM P
    """
    label_df = session.sql(labeling_sql).to_pandas()

    def clean_label(s: str) -> str:
        s = (s or "").strip().replace('"', "").replace("'", "")
        s = s.splitlines()[0].strip()
        return s[:MAX_LABEL_LEN] if len(s) > MAX_LABEL_LEN else s

    label_df["CLUSTER_LABEL"] = label_df["RAW_LABEL"].map(clean_label)
    label_df = label_df[["CLUSTER_ID", "CLUSTER_LABEL"]]

    # Merge back (now there's only ONE CLUSTER_LABEL)
    cluster_dict = tmp.merge(label_df, on="CLUSTER_ID", how="left")
else:
    cluster_dict["CLUSTER_LABEL"] = []

# Outlier row
outlier_row = pd.DataFrame([{
    "CLUSTER_ID": -1,
    "EXAMPLE_KEYWORDS": [],
    "CLUSTER_SIZE": int((labels == -1).sum()),
    "CLUSTER_LABEL": OUTLIER_LABEL,
}])
cluster_dict_full = pd.concat([cluster_dict, outlier_row], ignore_index=True)

# ✅ Normalize schema before write (prevents stray/suffixed columns)
cluster_dict_full = cluster_dict_full[["CLUSTER_ID", "CLUSTER_LABEL", "EXAMPLE_KEYWORDS", "CLUSTER_SIZE"]]

# Overwrite the destination table
session.sql(f"DROP TABLE IF EXISTS {CLUSTER_DICTIONARY}").collect()
session.write_pandas(
    cluster_dict_full.assign(
        EXAMPLE_KEYWORDS=lambda df: df["EXAMPLE_KEYWORDS"].apply(lambda xs: xs if isinstance(xs, list) else [])
    ),
    CLUSTER_DICTIONARY,
    auto_create_table=True,
    overwrite=True,
)

print("✅ Cluster dictionary & labels")
cluster_dict_full


In [ ]:
kw_clusters = kw_clusters.merge(cluster_dict_full[["CLUSTER_ID","CLUSTER_LABEL"]], on="CLUSTER_ID", how="left")

session.write_pandas(kw_clusters, KEYWORD_CLUSTERS, auto_create_table=True, overwrite=True)
print("✅ Keyword cluster assignments written.")
kw_clusters


In [ ]:
# 1) Bring original KEYWORDS JSON from source table
src = session.table(SOURCE_TABLE).select("SURVEY_ID", "KEYWORDS")

# 2) Join exploded keywords -> cluster labels
exploded = session.table(SURVEY_KEYWORDS_EXPLODED).select("SURVEY_ID","KEYWORD")
kwc = session.table(KEYWORD_CLUSTERS).select("KEYWORD","CLUSTER_LABEL")

# 3) Aggregate deduped labels per SURVEY_ID
session.sql(f"""
CREATE OR REPLACE TEMP VIEW V_SURVEY_LABELS AS
SELECT
  e.SURVEY_ID,
  ARRAY_DISTINCT(ARRAY_AGG(kc.CLUSTER_LABEL)) AS CLUSTER_LABELS
FROM {SURVEY_KEYWORDS_EXPLODED} e
JOIN {KEYWORD_CLUSTERS} kc
  ON e.KEYWORD = kc.KEYWORD
GROUP BY e.SURVEY_ID
""").collect()

# 4) Build the output table exactly as requested
session.sql(f"""
CREATE OR REPLACE TABLE {SURVEY_CLUSTER_OUTPUT} AS
SELECT
  s.SURVEY_ID,
  s.REVIEW_TEXT,
  s.KEYWORDS,
  COALESCE(v.CLUSTER_LABELS, ARRAY_CONSTRUCT()) AS CLUSTER_LABELS
FROM {SOURCE_TABLE} s
LEFT JOIN V_SURVEY_LABELS v
  ON s.SURVEY_ID = v.SURVEY_ID
""").collect()

print(f"✅ Final output table written: {SURVEY_CLUSTER_OUTPUT}")

# Preview final output
final_preview = session.table(SURVEY_CLUSTER_OUTPUT).limit(10).to_pandas()
final_preview


In [ ]:
select * from SURVEY_CLUSTER_OUTPUT limit 10;

### USE AI to label surveys (alternate to ML method)

In [ ]:
CREATE OR REPLACE TABLE SURVEY_CLUSTER_OUTPUT_ALT AS 
SELECT *,
AI_CLASSIFY(
  review_text,
  [
    {'label': 'Product Feature', 'description': 'This review specifically mentions something about the features or a characteristic of a lululemon product, good or bad.'},
    {'label': 'Fit and Sizing', 'description': 'This review mentions something about how the product fits, feels, or its size'},
    {'label': 'Price Value', 'description': 'This review mentions something regarding the price or value of the product'},
    {'label': 'Product Use', 'description': 'This review mentions something about how the product is being used like they are used for errands, fitness, yoga, etc'},
    {'label': 'Suggestion', 'description': 'This review provides a suggestion from customers on how to improve the product or experience'}
  ],
  {
    'task_description': 'Determine the appropriate topic labels for this lululemon product review',
    'output_mode': 'multi',
    'examples': [
      {
        'input': 'These exceeded my expectations! comfortable all day and great value.',
        'labels': ['Fit and Sizing', 'Price Value'],
        'explanation': 'the review mentions being comfortable which corresponds to Fit and Sizing and great value which corresponds to Price Value'
      },
      {
        'input': 'They work well but sizing runs a bit small.',
        'labels': ['Fit and Sizing', 'Price Value'],
        'explanation': 'the review mentions being comfortable which corresponds to Fit and Sizing and great value which corresponds to Price Value'
      }
    ]
  }) as AI_CLUSTER,
FROM SURVEY_CLUSTER_OUTPUT;

In [ ]:
select * from SURVEY_CLUSTER_OUTPUT_ALT limit 10;

In [ ]:
Select * from SURVEY_CLUSTER_OUTPUT_ALT
WHERE AI_FILTER(CONCAT('This review contains a suggestion from the customer about how the product or service can be improved: ', REVIEW_TEXT));

## Find sentiments of Surveys based on ASPECTS/Keyowrds and label

In [ ]:
CREATE OR REPLACE TABLE SENTIMENT_OUTPUT AS
SELECT SURVEY_ID, REVIEW_TEXT, CLUSTER_LABELS,
AI_SENTIMENT(REVIEW_TEXT, CLUSTER_LABELS) AS SENTIMENT
FROM SURVEY_CLUSTER_OUTPUT_ALT;

In [ ]:
SELECT * FROM SENTIMENT_OUTPUT limit 10;

In [ ]:
-- Explode sentiment categories, join to surveys, add normalized score + quarter keys
CREATE OR REPLACE VIEW DEMO_DB.CRM.SENTIMENT_EXPLODED_VW AS
SELECT
  so.SURVEY_ID,
  ps.PRODUCT_ID,
  ps.PRODUCT_NAME,
  DATE_TRUNC('QUARTER', ps.SURVEY_DATE)              AS QUARTER_START,
  YEAR(ps.SURVEY_DATE)                                AS SURVEY_YEAR,
  QUARTER(ps.SURVEY_DATE)                             AS SURVEY_QTR,
  f.value:name::string                                AS CATEGORY,
  LOWER(f.value:sentiment::string)                    AS SENTIMENT_LABEL,
  CASE LOWER(f.value:sentiment::string)
    WHEN 'positive' THEN 1.0::FLOAT
    WHEN 'neutral'  THEN 0.5::FLOAT
    WHEN 'unknown'  THEN 0.5::FLOAT
    WHEN 'negative' THEN 0.0::FLOAT
    ELSE NULL
  END                                                 AS SENTIMENT_SCORE
FROM DEMO_DB.CRM.SENTIMENT_OUTPUT so
JOIN DEMO_DB.CRM.LULULEMON_PRODUCT_SURVEY ps
  ON ps.SURVEY_ID = so.SURVEY_ID
, LATERAL FLATTEN(input => so.SENTIMENT:categories) f
WHERE ps.SURVEY_DATE IS NOT NULL AND
ps.review_text is not null AND
category <> 'overall';


In [ ]:
select * from SENTIMENT_EXPLODED_VW limit 10;

In [ ]:
import streamlit as st
st.title("Survey Sentiment")

# ---------------------------
# Overall sentiment: single bar chart (avg score per product, descending)
# ---------------------------
st.subheader("Overall Sentiment Score by Product (0–1, higher is more positive)")

overall_df = session.sql("""
  SELECT
    PRODUCT_NAME,
    AVG(CAST(SENTIMENT_SCORE AS FLOAT)) AS AVG_SCORE
  FROM DEMO_DB.CRM.SENTIMENT_EXPLODED_VW
  GROUP BY PRODUCT_NAME
  ORDER BY AVG_SCORE DESC
""").to_pandas()

if overall_df.empty:
  st.info("No data available.")
else:
  by_product = (
      overall_df
      .set_index("PRODUCT_NAME")["AVG_SCORE"]
      .sort_values(ascending=False)
  )
  
  st.bar_chart(by_product)
  st.dataframe(by_product, use_container_width=True)

# ---------------------------
# Product selector (single) — shown BELOW the overall chart
# ---------------------------
products_df = session.sql("""
  WITH base AS (
    SELECT DISTINCT PRODUCT_ID, PRODUCT_NAME
    FROM DEMO_DB.CRM.SENTIMENT_EXPLODED_VW
  )
  SELECT PRODUCT_ID, PRODUCT_NAME
  FROM (
    SELECT
      PRODUCT_ID,
      PRODUCT_NAME,
      ROW_NUMBER() OVER (PARTITION BY PRODUCT_NAME ORDER BY PRODUCT_ID) AS RN
    FROM base
  )
  WHERE RN = 1
  ORDER BY PRODUCT_NAME
""").to_pandas()


product_display = products_df["PRODUCT_NAME"].fillna(products_df["PRODUCT_ID"])
product_choice = st.selectbox(
    "Select a product",
    options=list(product_display),
    index=0 if len(product_display) else None
)

if len(products_df):
  selected_row = products_df.iloc[product_display[product_display == product_choice].index[0]]
  selected_product_id = selected_row["PRODUCT_ID"]
  selected_product_name = selected_row["PRODUCT_NAME"]
else:
  selected_product_id = None
  selected_product_name = None

# ---------------------------
# Product-specific: Top 5 positive labels (highest avg score)
# ---------------------------
col1, col2 = st.columns(2)

with col1:
  st.subheader(f"Top 5 Positive Labels for {selected_product_name or '(no selection)'} (highest avg score)")
  if selected_product_id:
    top_pos = session.sql(f"""
      SELECT
        CATEGORY,
        AVG(CAST(SENTIMENT_SCORE AS FLOAT)) AS AVG_SCORE
      FROM DEMO_DB.CRM.SENTIMENT_EXPLODED_VW
      WHERE PRODUCT_ID = '{selected_product_id}'
      GROUP BY CATEGORY
      ORDER BY AVG_SCORE DESC NULLS LAST, CATEGORY
      LIMIT 5
    """).to_pandas()

    # enforce order for the chart
    top_pos = top_pos.sort_values("AVG_SCORE", ascending=False)

    if top_pos.empty:
        st.info("No category data for this product.")
    else:
        st.bar_chart(top_pos.set_index("CATEGORY")["AVG_SCORE"])
        st.dataframe(top_pos, use_container_width=True)
  else:
    st.info("Select a product to see top positive labels.")

# ---------------------------
# Product-specific: Top 5 negative labels (lowest avg score)
# ---------------------------
with col2:
  st.subheader(f"Top 5 Negative Labels for {selected_product_name or '(no selection)'} (lowest avg score)")
  if selected_product_id:
    top_neg = session.sql(f"""
      SELECT
        CATEGORY,
        AVG(CAST(SENTIMENT_SCORE AS FLOAT)) AS AVG_SCORE
      FROM DEMO_DB.CRM.SENTIMENT_EXPLODED_VW
      WHERE PRODUCT_ID = '{selected_product_id}'
      GROUP BY CATEGORY
      ORDER BY AVG_SCORE ASC NULLS LAST, CATEGORY
      LIMIT 5
    """).to_pandas()

    if top_neg.empty:
      st.info("No category data for this product.")
    else:
      st.bar_chart(top_neg.set_index("CATEGORY")["AVG_SCORE"])
      st.dataframe(top_neg, use_container_width=True)
  else:
    st.info("Select a product to see top negative labels.")

# ---------------------------
# Product-specific: Per-category trend over time (avg score by quarter)
# ---------------------------
st.subheader(f"Per-Category Trend Over Time for {selected_product_name or '(no selection)'} (avg score per quarter)")

if selected_product_id:
  trend_df = session.sql(f"""
    WITH agg AS (
      SELECT
        CATEGORY,
        QUARTER_START,
        AVG(SENTIMENT_SCORE) AS AVG_SCORE
      FROM DEMO_DB.CRM.SENTIMENT_EXPLODED_VW
      WHERE PRODUCT_ID = '{selected_product_id}'
      GROUP BY CATEGORY, QUARTER_START
    ),
    ranked AS (
      -- Choose top 5 categories by number of time points to keep charts readable
      SELECT
        CATEGORY,
        COUNT(*) AS PTS,
        ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS RN
      FROM agg
      GROUP BY CATEGORY
    )
    SELECT a.CATEGORY, a.QUARTER_START, a.AVG_SCORE
    FROM agg a
    JOIN ranked r USING (CATEGORY)
    WHERE r.RN <= 5
    ORDER BY a.CATEGORY, a.QUARTER_START
  """).to_pandas()

  if trend_df.empty:
    st.info("No category trend data for this product.")
  else:
    cats = list(trend_df["CATEGORY"].unique())
    tabs = st.tabs(cats)
    for tab, cat in zip(tabs, cats):
      with tab:
        df_cat = trend_df[trend_df["CATEGORY"] == cat].set_index("QUARTER_START").sort_index()
        st.line_chart(df_cat["AVG_SCORE"])
        st.dataframe(df_cat.reset_index(), use_container_width=True)
else:
  st.info("Select a product to see per-category trends.")
